In [ ]:
from Binaries import *

In [ ]:
q_peak = np.array([GLOBAL.q_peak for _ in range(3)])
q_charge = np.array([GLOBAL.q_charge for _ in range(3)])

EventsADC = EventGenerator(["19_19.5"], real_background = False, sigma = 0, split = 1, apply_downsampling = False, floor_trace = False)
EventsADC.files = ["/cr/tempdata01/filip/QGSJET-II/COMPARE/ADC/" + file for file in os.listdir("/cr/tempdata01/filip/QGSJET-II/COMPARE/ADC")]

q_peak = np.array([1, 1, 1])
q_charge = np.array([GLOBAL.q_charge / GLOBAL.q_peak for _ in range(3)])

EventsVEM = EventGenerator(["19_19.5"], real_background = False, sigma = 0, split = 1, q_peak = q_peak, q_charge = q_charge, apply_downsampling = False, floor_trace = False)
EventsVEM.files = ["/cr/tempdata01/filip/QGSJET-II/COMPARE/VEM/" + file for file in os.listdir("/cr/tempdata01/filip/QGSJET-II/COMPARE/VEM")]

In [ ]:
adc_dir = "/cr/tempdata01/filip/QGSJET-II/COMPARE/ADC"
vem_dir = "/cr/tempdata01/filip/QGSJET-II/COMPARE/VEM"

print([file for file in os.listdir(adc_dir) if file not in os.listdir(vem_dir)])

In [ ]:
start_time = perf_counter_ns()
image = np.zeros((1001, 2048))

for i in range(EventsVEM.__len__()):

    progress_bar(i, EventsVEM.__len__(), start_time)

    tracesVEM = EventsVEM.__getitem__(i)

    try:
        j = EventsADC.files.index("/cr/tempdata01/filip/QGSJET-II/COMPARE/ADC/" + EventsVEM.files[i].split('/')[-1])
    except ValueError: print("Cache miss", EventsVEM.files[i].split('/')[-1])

    tracesADC = EventsADC.__getitem__(j)

    for traceVEM, traceADC in zip(tracesVEM, tracesADC):

        diff = np.clip((traceVEM.Signal - traceADC.Signal / GLOBAL.q_peak) / traceVEM.Signal, -1, 1)

        for pmt in diff:
            for i, bin in enumerate(pmt):
                if np.isnan(bin): continue
                image[int(bin * 500) + 500][i] += 1


    if i == 10: raise StopIteration

In [ ]:
plt.imshow(image, origin = "lower")
_, _ = plt.yticks(np.arange(0, 1001, 100), [f"{b:.1f}" for b in np.arange(0, 1001, 100)/500 - 1])
plt.ylabel(r"PMT$_\mathrm{VEM}$ - $\frac{\mathrm{PMT}_\mathrm{ADC}}{q_\mathrm{Peak}}$ / VEM")

In [ ]:
print(image[0][:])

In [ ]:
traceVEM.__plot__()
traceADC.__plot__()

In [ ]:
print(diff)